<a href="https://colab.research.google.com/github/tashkinovnet/Home-Task/blob/Hometask_7.ipynb/Hometask_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Hometask 7**

###Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета. Добейтесь того, чтобы хотя бы несколько тем были явно интерпретируемы, например, как в примерах ниже.

Примеры топ-10 слов из некотрых тегов, которые получаются после применения LDA:
* ['god', 'jesus', 'believe', 'life', 'bible', 'christian', 'world', 'church', 'word', 'people'] - эта группа явно соотносится с soc.religion.christian
* ['drive', 'card', 'hard', 'bit', 'disk', 'scsi', 'memory', 'speed', 'mac', 'video'] - эту группу можно соотнести с темами 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware'
* ['game',	'games',	'hockey',	'league',	'play',	'players',	'season',	'team',	'teams',	'win'] - тема rec.sport.hockey

Советы:
* модель будет сходится лучше и быстрее, если уменьшить размер словаря за счет отсеивания общеупотребительных слов и редких слов. Управлять размером словаря можно с помощью параметров min_df (отсеивает слова по минимальной частоте встречаемости) и max_df (отсеивает слова по максимальной частоте встречаемости) в CountVectorizer.
* параметры $\alpha$, $\beta$ можно, для начала, положить единицами
* после 100 итераций можно ожидать хорошего распределения по темам. Если этого не происходит и в темах мешинина - проверяйте код и оптимизируйте словарь
* на примере третьей темы видно, что у нас встречаются разные формы одного и того же слова. С помощью процедур stemming и lemmatization можно привести слова к общей форме и объединить близкие по значению

In [9]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
documents = newsgroups_train.data
labels = newsgroups_train.target
label_names = newsgroups_train.target_names

vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2,
    stop_words='english',
    max_features=1000
)
X = vectorizer.fit_transform(documents)

n_topics = 20
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=100, random_state=42)
lda.fit(X)

def print_top_words(model, feature_names, n_top_words):
    topics = []
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics.append(top_words)
        print(f"Topic #{topic_idx + 1}: {', '.join(top_words)}")
    return topics

feature_names = vectorizer.get_feature_names_out()
topics = print_top_words(lda, feature_names, 10)

# Категории тем
topic_assignments = lda.transform(X).argmax(axis=1)
topic_to_labels = {i: [] for i in range(n_topics)}

# Группировка по темам
for i, topic in enumerate(topic_assignments):
    topic_to_labels[topic].append(labels[i])

for topic_idx, label_list in topic_to_labels.items():
    label_counts = np.bincount(label_list, minlength=len(label_names))
    top_label_idx = label_counts.argmax()
    print(f"Topic #{topic_idx + 1}: {label_names[top_label_idx]} ({label_counts[top_label_idx]} documents)")
    print(f"Top words: {', '.join(topics[topic_idx])}\n")


Topic #1: key, chip, encryption, keys, clipper, privacy, use, security, public, government
Topic #2: god, jesus, bible, christian, believe, church, faith, people, christians, life
Topic #3: file, windows, use, window, program, using, output, files, set, entry
Topic #4: said, people, armenian, did, didn, went, armenians, came, know, told
Topic #5: game, team, year, games, play, season, hockey, players, league, win
Topic #6: available, data, software, image, ftp, version, graphics, information, based, program
Topic #7: thanks, know, does, help, like, mail, looking, hi, need, advance
Topic #8: mr, president, israel, israeli, stephanopoulos, going, think, know, people, jobs
Topic #9: government, law, people, state, right, rights, public, laws, military, federal
Topic #10: drive, card, scsi, disk, mac, hard, video, pc, use, bit
Topic #11: 10, 11, 25, 14, 17, 12, 15, 16, db, 13
Topic #12: new, power, time, years, year, high, used, cost, low, air
Topic #13: space, 00, nasa, launch, earth, sat